# The Data

In [1]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
len(documents)

1186018

In [3]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


# Data Pre-processing

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\רועי\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatize example

In [6]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


### Stemmer Example

In [7]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [8]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [9]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [10]:
processed_docs = documents['headline_text'].map(preprocess)

In [11]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

# Bag of words on the dataset

In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [13]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3567, 1), (3568, 1)]

In [16]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3567 ("compulsori") appears 1 time.
Word 3568 ("ratepay") appears 1 time.


# TF-IDF

In [17]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [18]:
corpus_tfidf = tfidf[bow_corpus]

In [19]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]


# Running LDA using Bag of Words

In [20]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [21]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"hous" + 0.022*"south" + 0.020*"north" + 0.016*"miss" + 0.015*"bushfir" + 0.013*"interview" + 0.012*"west" + 0.012*"hospit" + 0.011*"coast" + 0.010*"island"
Topic: 1 
Words: 0.031*"kill" + 0.023*"shoot" + 0.020*"dead" + 0.020*"attack" + 0.019*"polic" + 0.018*"protest" + 0.014*"offic" + 0.013*"assault" + 0.011*"michael" + 0.011*"bank"
Topic: 2 
Words: 0.056*"australia" + 0.045*"australian" + 0.026*"world" + 0.017*"test" + 0.017*"canberra" + 0.013*"win" + 0.011*"final" + 0.011*"open" + 0.011*"farm" + 0.010*"return"
Topic: 3 
Words: 0.031*"polic" + 0.029*"charg" + 0.026*"court" + 0.025*"death" + 0.024*"murder" + 0.020*"woman" + 0.017*"face" + 0.017*"alleg" + 0.016*"crash" + 0.013*"jail"
Topic: 4 
Words: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"concern" + 0.012*"climat" + 0.011*"flood" + 0.010*"fear" + 0.010*"storm"
Topic: 5 
Words: 0.022*"market" + 0.019*"news" + 0.018*"women" + 0.017*"live" + 0.016*"tasmania" + 0.013*"ri

Cool! Can you distinguish different topics using the words in each topic and their corresponding weights?

# Running LDA using TF-IDF

In [22]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [23]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.022*"news" + 0.014*"crash" + 0.012*"market" + 0.009*"miss" + 0.009*"die" + 0.008*"friday" + 0.008*"search" + 0.007*"violenc" + 0.006*"dead" + 0.006*"rescu"
Topic: 1 Word: 0.012*"rural" + 0.011*"govern" + 0.007*"elect" + 0.007*"nation" + 0.006*"feder" + 0.006*"say" + 0.006*"fund" + 0.005*"labor" + 0.005*"plan" + 0.005*"morrison"
Topic: 2 Word: 0.020*"countri" + 0.015*"hour" + 0.009*"turnbul" + 0.009*"health" + 0.008*"bushfir" + 0.007*"mental" + 0.006*"beach" + 0.006*"decemb" + 0.006*"explain" + 0.006*"insid"
Topic: 3 Word: 0.017*"donald" + 0.009*"scott" + 0.008*"wall" + 0.008*"sport" + 0.008*"street" + 0.006*"septemb" + 0.006*"octob" + 0.006*"novemb" + 0.005*"facebook" + 0.005*"toni"
Topic: 4 Word: 0.011*"australia" + 0.011*"world" + 0.010*"final" + 0.009*"leagu" + 0.008*"monday" + 0.008*"live" + 0.008*"wednesday" + 0.008*"cricket" + 0.007*"david" + 0.007*"juli"
Topic: 5 Word: 0.014*"drum" + 0.013*"interview" + 0.010*"royal" + 0.009*"tuesday" + 0.008*"korea" + 0.008*"mi

# Classification of the topics

In [24]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [25]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.457031786441803	 
Topic: 0.020*"donald" + 0.014*"nation" + 0.014*"farmer" + 0.014*"plan" + 0.013*"rural" + 0.013*"time" + 0.013*"council" + 0.013*"indigen" + 0.011*"school" + 0.011*"commiss"

Score: 0.4429357051849365	 
Topic: 0.031*"govern" + 0.020*"warn" + 0.019*"feder" + 0.015*"countri" + 0.015*"fund" + 0.014*"claim" + 0.014*"life" + 0.012*"say" + 0.012*"health" + 0.011*"stori"

Score: 0.012506910599768162	 
Topic: 0.035*"elect" + 0.018*"water" + 0.018*"state" + 0.016*"tasmanian" + 0.012*"labor" + 0.011*"liber" + 0.011*"morrison" + 0.011*"leader" + 0.010*"parti" + 0.010*"campaign"

Score: 0.012504234910011292	 
Topic: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"concern" + 0.012*"climat" + 0.011*"flood" + 0.010*"fear" + 0.010*"storm"

Score: 0.012503559701144695	 
Topic: 0.022*"hous" + 0.022*"south" + 0.020*"north" + 0.016*"miss" + 0.015*"bushfir" + 0.013*"interview" + 0.012*"west" + 0.012*"hospit" + 0.011*"coast" + 0.010*"island"



Our test document has the highest probability to be part of the topic on the top.

# Performance evaluation by classifying sample document using LDA TF-IDF model

In [26]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5334686040878296	 
Topic: 0.012*"rural" + 0.011*"govern" + 0.007*"elect" + 0.007*"nation" + 0.006*"feder" + 0.006*"say" + 0.006*"fund" + 0.005*"labor" + 0.005*"plan" + 0.005*"morrison"

Score: 0.19596916437149048	 
Topic: 0.017*"donald" + 0.009*"scott" + 0.008*"wall" + 0.008*"sport" + 0.008*"street" + 0.006*"septemb" + 0.006*"octob" + 0.006*"novemb" + 0.005*"facebook" + 0.005*"toni"

Score: 0.18301677703857422	 
Topic: 0.020*"countri" + 0.015*"hour" + 0.009*"turnbul" + 0.009*"health" + 0.008*"bushfir" + 0.007*"mental" + 0.006*"beach" + 0.006*"decemb" + 0.006*"explain" + 0.006*"insid"

Score: 0.012507101520895958	 
Topic: 0.014*"drum" + 0.013*"interview" + 0.010*"royal" + 0.009*"tuesday" + 0.008*"korea" + 0.008*"michael" + 0.007*"commiss" + 0.007*"kill" + 0.006*"presid" + 0.006*"russia"

Score: 0.012507045641541481	 
Topic: 0.009*"thursday" + 0.007*"price" + 0.006*"dairi" + 0.006*"jam" + 0.006*"fiji" + 0.006*"post" + 0.006*"disabl" + 0.006*"asylum" + 0.005*"territori" + 0.005*

# Testing model on unseen document

In [27]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4890992343425751	 Topic: 0.022*"market" + 0.019*"news" + 0.018*"women" + 0.017*"live" + 0.016*"tasmania"
Score: 0.2112114429473877	 Topic: 0.035*"elect" + 0.018*"water" + 0.018*"state" + 0.016*"tasmanian" + 0.012*"labor"
Score: 0.18295393884181976	 Topic: 0.031*"polic" + 0.029*"charg" + 0.026*"court" + 0.025*"death" + 0.024*"murder"
Score: 0.016677169129252434	 Topic: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker"
Score: 0.016677040606737137	 Topic: 0.031*"govern" + 0.020*"warn" + 0.019*"feder" + 0.015*"countri" + 0.015*"fund"
Score: 0.01667623408138752	 Topic: 0.022*"hous" + 0.022*"south" + 0.020*"north" + 0.016*"miss" + 0.015*"bushfir"
Score: 0.01667623408138752	 Topic: 0.031*"kill" + 0.023*"shoot" + 0.020*"dead" + 0.020*"attack" + 0.019*"polic"
Score: 0.01667623408138752	 Topic: 0.056*"australia" + 0.045*"australian" + 0.026*"world" + 0.017*"test" + 0.017*"canberra"
Score: 0.01667623408138752	 Topic: 0.020*"donald" + 0.014*"nation" + 0.014*"far